# 🐱🐶 Cat vs Dog Classification - Interactive Demo

This notebook demonstrates the complete workflow for training and evaluating a cat vs dog classifier with advanced features.

## Features:
- Custom CNN with attention mechanisms
- Advanced data augmentation
- Grad-CAM visualization
- Model interpretability

## 1. Setup and Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import cv2
from glob import glob
from tqdm.notebook import tqdm

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Check GPU
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Import project modules
sys.path.append('..')
from models.architectures import build_custom_cnn, build_efficientnet_transfer
from utils.augmentation import AdvancedAugmentation, MixupCutmix
from utils.gradcam import GradCAM
from utils.training import TrainingConfig, TrainingPipeline

## 2. Model Architecture Exploration

In [ ]:
# Build and visualize the custom CNN architecture
model = build_custom_cnn(input_shape=(224, 224, 3))
model.summary()

print(f"\nTotal parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.reduce_prod(v.shape) for v in model.trainable_variables]):,}")

In [ ]:
# Visualize model architecture
keras.utils.plot_model(
    model,
    to_file='model_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    dpi=96
)

## 3. Data Augmentation Demo

In [ ]:
# Load a sample image
sample_image_path = '../data/cat.0.jpg'  # Adjust path as needed

if os.path.exists(sample_image_path):
    sample_image = cv2.imread(sample_image_path)
    sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
else:
    # Create a dummy image if no real data available
    sample_image = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
    print("Using dummy image for demonstration")

# Test different augmentation levels
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('Data Augmentation Examples', fontsize=16, fontweight='bold')

augmentation_levels = ['light', 'medium', 'heavy']

for row_idx, level in enumerate(augmentation_levels):
    aug = AdvancedAugmentation(img_size=224, augmentation_level=level)
    
    for col_idx in range(4):
        if col_idx == 0 and row_idx == 0:
            # Show original in first cell
            axes[row_idx, col_idx].imshow(cv2.resize(sample_image, (224, 224)))
            axes[row_idx, col_idx].set_title('Original', fontweight='bold')
        else:
            # Show augmented versions
            augmented = aug(sample_image.copy())
            axes[row_idx, col_idx].imshow(augmented)
            if col_idx == 0:
                axes[row_idx, col_idx].set_title(f'{level.capitalize()} Aug', fontweight='bold')
        
        axes[row_idx, col_idx].axis('off')

plt.tight_layout()
plt.show()

## 4. Mixup and CutMix Demo

In [ ]:
# Create sample batch
batch_images = np.random.random((4, 224, 224, 3)).astype(np.float32)
batch_labels = np.array([0, 1, 0, 1], dtype=np.float32)

# Apply Mixup
mixup = MixupCutmix(alpha=0.2, cutmix_prob=0.0)
mixed_images, mixed_labels = mixup(batch_images.copy(), batch_labels.copy())

# Apply CutMix
cutmix = MixupCutmix(alpha=0.2, cutmix_prob=1.0)
cut_images, cut_labels = cutmix(batch_images.copy(), batch_labels.copy())

# Visualize
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('Mixup and CutMix Augmentation', fontsize=16, fontweight='bold')

for i in range(4):
    # Original
    axes[0, i].imshow(batch_images[i])
    axes[0, i].set_title(f'Original (Label: {batch_labels[i]:.1f})')
    axes[0, i].axis('off')
    
    # Mixup
    axes[1, i].imshow(mixed_images[i])
    axes[1, i].set_title(f'Mixup (Label: {mixed_labels[i]:.2f})')
    axes[1, i].axis('off')
    
    # CutMix
    axes[2, i].imshow(cut_images[i])
    axes[2, i].set_title(f'CutMix (Label: {cut_labels[i]:.2f})')
    axes[2, i].axis('off')

plt.tight_layout()
plt.show()

## 5. Training Configuration

In [ ]:
# Create training configuration
config = TrainingConfig()

# Modify for quick demo (reduce epochs)
config.epochs = 10  # Use 50+ for real training
config.batch_size = 32
config.augmentation_level = 'medium'

print("Training Configuration:")
print("=" * 50)
for key, value in config.to_dict().items():
    print(f"{key:25s}: {value}")
print("=" * 50)

## 6. Learning Rate Schedule Visualization

In [ ]:
from utils.training import CosineDecayWithWarmup

# Create learning rate schedule
total_steps = 1000
warmup_steps = 100

lr_schedule = CosineDecayWithWarmup(
    initial_learning_rate=1e-3,
    decay_steps=total_steps - warmup_steps,
    warmup_steps=warmup_steps,
    alpha=0.01
)

# Generate learning rates
steps = np.arange(0, total_steps)
learning_rates = [lr_schedule(step).numpy() for step in steps]

# Plot
plt.figure(figsize=(12, 6))
plt.plot(steps, learning_rates, linewidth=2)
plt.axvline(x=warmup_steps, color='r', linestyle='--', label='Warmup End', alpha=0.7)
plt.xlabel('Training Step', fontsize=12)
plt.ylabel('Learning Rate', fontsize=12)
plt.title('Cosine Decay Learning Rate Schedule with Warmup', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Model Training (Demo)

**Note**: This section requires actual cat and dog images. For demonstration purposes, we'll show the training setup. For full training, use `main_train.py`.

In [ ]:
# Example training code (requires real data)
# Uncomment and modify paths as needed

# from main_train import prepare_data, train_model
# 
# # Prepare data
# train_paths, train_labels, val_paths, val_labels, test_paths, test_labels = \
#     prepare_data('../data', img_size=224)
# 
# # Train model
# model, history, test_results = train_model(
#     data_dir='../data',
#     model_type='custom',
#     config=config,
#     output_dir='../outputs'
# )

print("Training code ready. Use main_train.py for full training.")

## 8. Grad-CAM Visualization Demo

In [ ]:
# Load a pre-trained model (or use the one just trained)
# model_path = '../outputs/best_custom_model.keras'
# model = keras.models.load_model(model_path)

# For demo, we'll use the untrained model
model = build_custom_cnn(input_shape=(224, 224, 3))

# Create Grad-CAM object
grad_cam = GradCAM(model)

# Load sample image
if os.path.exists(sample_image_path):
    test_image = cv2.imread(sample_image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
else:
    test_image = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)

# Preprocess
test_image_resized = cv2.resize(test_image, (224, 224))
test_image_processed = test_image_resized.astype(np.float32) / 255.0

# Generate Grad-CAM visualization
fig = grad_cam.visualize(
    image=test_image_processed,
    original_image=test_image_resized,
    save_path='gradcam_demo.png'
)
plt.show()

## 9. Model Inference Demo

In [ ]:
def predict_image(model, image_path):
    """Make prediction on a single image"""
    # Load image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    original = img.copy()
    
    # Preprocess
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0
    
    # Predict
    prediction = model.predict(np.expand_dims(img, axis=0), verbose=0)[0][0]
    
    # Interpret
    if prediction > 0.5:
        pred_class = "Dog"
        confidence = prediction
    else:
        pred_class = "Cat"
        confidence = 1 - prediction
    
    # Visualize
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.resize(original, (224, 224)))
    plt.title(f'Prediction: {pred_class} (Confidence: {confidence:.2%})', 
             fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()
    
    return pred_class, confidence

# Example usage (requires trained model)
# pred_class, confidence = predict_image(model, sample_image_path)
print("Inference function ready. Use with trained model.")

## 10. Performance Analysis

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes=['Cat', 'Dog']):
    """Plot confusion matrix"""
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.show()

# Example (requires predictions)
# y_true = [0, 0, 1, 1, 0, 1]
# y_pred = [0, 0, 1, 0, 0, 1]
# plot_confusion_matrix(y_true, y_pred)

print("Confusion matrix function ready.")

## Summary

This notebook demonstrates:

1. ✅ Custom CNN architecture with attention mechanisms
2. ✅ Advanced data augmentation techniques
3. ✅ Mixup and CutMix implementations
4. ✅ Learning rate scheduling
5. ✅ Grad-CAM visualization for interpretability
6. ✅ Complete training and inference pipeline

### Next Steps:

1. **Download Data**: Get the Kaggle Dogs vs Cats dataset
2. **Train Model**: Use `main_train.py` for full training
3. **Evaluate**: Analyze performance on test set
4. **Deploy**: Launch web interface with `web_interface.py`

### Expected Results:

With proper training:
- **Accuracy**: 98-99%
- **AUC**: 0.99+
- **Training Time**: 2-4 hours on modern GPU

### References:

- Kaggle Dogs vs Cats: https://www.kaggle.com/c/dogs-vs-cats
- Grad-CAM Paper: https://arxiv.org/abs/1610.02391
- Mixup Paper: https://arxiv.org/abs/1710.09412
- CutMix Paper: https://arxiv.org/abs/1905.04899